In [4]:
import torch
from pathlib import Path
import pandas as pd
from collections import Counter
from Bio import SeqIO
import yaml

def get_pep_length(pep_fasta_path):
    records = list(SeqIO.parse(pep_fasta_path, "fasta"))
    pep_length = len(str(records[0].seq))
    return pep_length

def ncaa_num(yaml_file):
    with open(yaml_file, 'r') as file:
        config = yaml.safe_load(file)
    chain_num = 0
    for chain in config['sm_inputs']:
        chain_num += 1
    return chain_num

root_path = Path("/home/light/mqy/ncaa/data/ncaa/cyc/monomer/head_to_tail")
for subdir in root_path.iterdir():
    if subdir.is_dir():
        name = subdir.name
        # if "4llbC" in name:
        #     continue
        pt = subdir / f"{name}_aux_ckpt192.pt"
        pep_fasta_path = subdir / f"{name}-pep.fasta"
        yaml_file = subdir / "config_aa.yaml"
        try:
            ncaa = ncaa_num(yaml_file)
            pep_length = get_pep_length(pep_fasta_path)
            pep_aa = pep_length - ncaa

            data = torch.load(pt)
            plddt = data["mean_plddt"]
            pep_plddt = ((data["plddts"][0][:pep_aa-ncaa].sum()+data["plddts"][0][-ncaa:].sum())/pep_length).item()
            print(f"{name}:{pep_plddt:.3f}")
        except Exception as e:
            print(f"{name}:{str(e)}")


2N8E:0.840
2M2S:0.400
2KNN:0.855
2MSQ:0.841
4M6E:0.336
2J15:0.556
2M2H:0.492
2M1P:0.716
2M2X:0.138
2M2G:0.773
1T9E:0.647
